# Testing Dataset UI Modules

Test out the newly written data_interface modules, `data_table` and `retrieve_uv_data`

## Data Table

In [ ]:
from pathlib import Path

from data_interface import data_table, retrieve_uv_data
from hplc_dad_plots import plot_3d_line

p = Path("/Users/jonathan/0_jono_data")

print(p)

In [ ]:
df = data_table(p)

df = df.sort_values(by="acquisition_date", ascending=False)

df.head()

Now pull a particular dataset and extract the uv data

In [ ]:
data = df.loc["2021-DEBORTOLI-CABERNET-MERLOT_HALO.D"]["data"]

uv_data = retrieve_uv_data(data)

uv_data.head()

It would be useful to drop unwanted or unpopulated wavelengths. Since wavelengths are stored column by column, to drop a range we'd need to convert to a long format then institute a logical mask, then swap back to wide.

## Dropping Unpopulated or Baseline-Biased Wavelengths

In [ ]:
# First I have to melt the dataframe so that the wavelengths can be treated as values rather than column names.

uv_data_melt = uv_data.melt(id_vars="mins", var_name="nm", value_name="mAU")

# then
uv_data_melt = uv_data_melt[(uv_data_melt["nm"] > 210) & (uv_data_melt["nm"] < 255)]
uv_data_melt.head()

Now to reset it back to wide format to be compatible with the plotting function.

In [ ]:
uv_data_211 = uv_data_melt.pivot(columns=["nm"], index="mins", values="mAU")

uv_data_211 = uv_data_211.reset_index(names="mins")

uv_data_211.head()

In [ ]:
plot_3d_line(uv_data_211, plot_title=data.name)

We're looking like we've got a good interface going now. next step will be to:

- Integrate sequences into the table.
- Get a dashboard going with Panel.
- Get some descriptive measurements going through scipy.
- get some statistical data going.